<a href="https://colab.research.google.com/github/ujjalkumarmaity/research-paper-implementation/blob/main/GPT-2/fine-tune/Fine_tune_GPT_2_shakespeare_Poem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install accelerate -U

In [2]:
%%capture
!pip install datasets

# Prepare Dataset

In [3]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-01-20 16:44:02--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2024-01-20 16:44:02 (19.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [5]:
with open('input.txt','r',encoding='utf-8') as f:
    data = f.read()
def save_dataset(data,path):
    with open(path , 'w') as f:
        f.write(data)
len_data = len(data)
train,test = data[:int(len_data*0.9)],data[int(len_data*0.9):]
save_dataset(train,'train.txt')
save_dataset(test,'test.txt')

In [6]:
len(data)

1115394

In [7]:
from transformers import TextDataset,DataCollatorForLanguageModeling,AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')
train_data = TextDataset(tokenizer = tokenizer,
                         file_path = 'train.txt',
                         block_size = 128)
test_data = TextDataset(tokenizer = tokenizer,
                         file_path = 'test.txt',
                         block_size = 128)
# Data collator used for language modeling. Inputs are dynamically padded to
# the maximum length of a batch if they are not all of the same length.
data_collector = DataCollatorForLanguageModeling(tokenizer  = tokenizer,
                                                 mlm = False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (301966 > 1024). Running this sequence through the model will result in indexing errors


In [8]:
test_data[0]

tensor([   30,   198,   198, 28934,  8895,    46,    25,   198, 10248,  2146,
          808,    11, 12250, 18226, 12523,    13,   198,   198,    33,  2969,
           51,  1797,  5603,    25,   198, 10248,  2146,   808,    11, 12250,
          402,  2787,   952,    13,   198, 13482,  3613,   345,    11, 28527,
            0,   198,   198, 47731,    49,    52,  3398,  9399,    25,   198,
         1870,   345,    11,   922, 15967,     0,  1736,   323,    11,   423,
          345,   407,   257,  4957,   198, 14134,  1549, 18341,   283,  1437,
           11,  3148,   290, 41276,    30,   198,   198,    33,  2969,    51,
         1797,  5603,    25,   198,    40,   423,   257,  4957,    11, 15967,
           11,  1444, 18341,   283,  1437,    13,   198,   198, 28934,  8895,
           46,    25,   198,  1639,   389,  1165, 19861,    25,   467,   284,
          340, 39164,    13,   198,   198, 47731,    49,    52,  3398,  9399,
           25,   198,  1639,  2642,   502,    11,  5865,  1504])

# Initilize TrainingArguments

In [11]:
from transformers import AutoModel,AutoModelWithLMHead,Trainer,TrainingArguments
model = AutoModelWithLMHead.from_pretrained('gpt2')
train_arg = TrainingArguments(output_dir = './gpt2-fine-tune',
                              overwrite_output_dir = True,
                              do_eval  = True,
                              per_device_train_batch_size = 32,
                              per_device_eval_batch_size = 64,
                              num_train_epochs = 10,
                              eval_steps = 200,
                              warmup_steps=200)
trainer = Trainer(
    model = model,
    args = train_arg,
    data_collator = data_collector,
    train_dataset = train_data,
    eval_dataset = test_data
)

# Train and Save the Model

In [12]:
trainer.train()

Step,Training Loss
500,3.316800


TrainOutput(global_step=740, training_loss=3.240812229465794, metrics={'train_runtime': 878.3403, 'train_samples_per_second': 26.857, 'train_steps_per_second': 0.842, 'total_flos': 1540969758720000.0, 'train_loss': 3.240812229465794, 'epoch': 10.0})

In [13]:
model.save_pretrained('./gpt2-fine-tune')
tokenizer.save_pretrained('./gpt2-fine-tune')

('./gpt2-fine-tune/tokenizer_config.json',
 './gpt2-fine-tune/special_tokens_map.json',
 './gpt2-fine-tune/vocab.json',
 './gpt2-fine-tune/merges.txt',
 './gpt2-fine-tune/added_tokens.json',
 './gpt2-fine-tune/tokenizer.json')

# Test the model

In [14]:
from transformers import pipeline
model = pipeline('text-generation',model='./gpt2-fine-tune',tokenizer='./gpt2-fine-tune')

In [25]:
print(model("First Citizen",max_length=100, pad_token_id=50256, num_return_sequences=1)[0]['generated_text'])

First Citizen:
It's true.

CLARENCE:
It is.

MENENIUS:
Well, 'twas by trial,--

SICINIUS:
No matter how I would play the officer, I did not believe him as
necessary a man as I found him worthy of
standing and speaking. We spoke by chance not of what he did
against the general, but of what he did by himself
against the general's


In [27]:
print(model("If I must not",max_length=100, pad_token_id=50256, num_return_sequences=1)[0]['generated_text'])

If I must not make her love one, bethink me
That she will soon have in me kindred.

ISABELLA:
The queen is married; we know 'tis in deed.

ANGELO:
I am in a passion,
And do not wish her to lose her husband's.

ISABELLA:
And in that which I do intend
To take.

ANGELO:
Come, have you with you,


In [28]:
from transformers import AutoTokenizer,AutoModelWithLMHead
tokenizer = AutoTokenizer.from_pretrained('./gpt2-fine-tune')
model = AutoModelWithLMHead.from_pretrained('./gpt2-fine-tune')

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [30]:
input_ids = tokenizer("First Citizen",return_tensors='pt').input_ids

In [36]:
out = model.generate(input_ids,max_length=100,pad_token_id=50256)
print(tokenizer.batch_decode(out)[0])

First Citizen:
I am a gentleman, sir; and I am a gentleman
To whom you have given your consent.

First Citizen:
I am a gentleman to you, sir; and I am a gentleman
To whom you have given your consent.

Second Citizen:
I am a gentleman to you, sir; and I am a gentleman
To whom you have given your consent.

First Citizen:
I am a gentleman to you, sir; and I
